# Python for Webscraping
* SOC 590: Big Data and Population Processes
* 17th October 2016

## Tutorial 3: Webscraping and pre-processing 

* import relevant modules
* standard library modules:
    * [os](https://docs.python.org/3/library/os.html)
    * [urllib](https://docs.python.org/3/library/urllib.html)
    * [webbrowser](https://docs.python.org/3/library/webbrowser.html)
* open source modules:
    * [pandas](http://pandas.pydata.org)
    * [Beautiful Soup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)

In [1]:
import os
import urllib
import webbrowser
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
url = 'http://www.pro-football-reference.com/years/2015/passing.htm'
webbrowser.open_new_tab(url)

True

In [3]:
# The url we will be scraping
url_2015 = "http://www.pro-football-reference.com/years/2015/passing.htm"

# get the html
html = urllib.request.urlopen(url_2015)

# create the BeautifulSoup object
soup = BeautifulSoup(html, "lxml")

## Scraping the Column Headers

The column headers we need for our `DataFrame` are found in the second row of column headers PFR table.  We will will scrape those and add two additional columns headers for the two additional player page links.

In [4]:
# Extract the necessary values for the column headers from the table
# and store them as a list
column_headers = [th.getText() for th in soup.findAll('th', limit=30)]
column_headers = [s for s in column_headers if len(s) != 0]
column_headers = column_headers[1:]
column_headers

['Tm',
 'Age',
 'Pos',
 'G',
 'GS',
 'QBrec',
 'Cmp',
 'Att',
 'Cmp%',
 'Yds',
 'TD',
 'TD%',
 'Int',
 'Int%',
 'Lng',
 'Y/A',
 'AY/A',
 'Y/C',
 'Y/G',
 'Rate',
 'QBR',
 'Sk',
 'Yds',
 'NY/A',
 'ANY/A',
 'Sk%',
 '4QC',
 'GWD']

In [5]:
len(column_headers)

28

## Scraping the Data

In [6]:
# The data is found within the table rows
# We want the elements from the 3rd row and on
table_rows = soup.find_all("tr")[1:]
table_rows

[<tr><th class="right " csk="1" data-stat="ranker" scope="row">1</th><td class="left " csk="Rivers,Philip" data-append-csv="RivePh00" data-stat="player"><a href="/players/R/RivePh00.htm">Philip Rivers</a></td><td class="left " data-stat="team"><a href="/teams/sdg/2015.htm" title="San Diego Chargers">SDG</a></td><td class="right " data-stat="age">34</td><td class="left " data-stat="pos">QB</td><td class="right " data-stat="g">16</td><td class="right " data-stat="gs">16</td><td class="right " csk="0.25000" data-stat="qb_rec">4-12-0</td><td class="right " data-stat="pass_cmp">437</td><td class="right " data-stat="pass_att">661</td><td class="right " data-stat="pass_cmp_perc">66.1</td><td class="right " data-stat="pass_yds">4792</td><td class="right " data-stat="pass_td">29</td><td class="right " data-stat="pass_td_perc">4.4</td><td class="right " data-stat="pass_int">13</td><td class="right " data-stat="pass_int_perc">2.0</td><td class="right " data-stat="pass_long">80</td><td class="righ

Note that `table_rows` is a list of tag elements.

In [7]:
type(table_rows)

list

In [8]:
table_rows[0] # take a look at the first row

<tr><th class="right " csk="1" data-stat="ranker" scope="row">1</th><td class="left " csk="Rivers,Philip" data-append-csv="RivePh00" data-stat="player"><a href="/players/R/RivePh00.htm">Philip Rivers</a></td><td class="left " data-stat="team"><a href="/teams/sdg/2015.htm" title="San Diego Chargers">SDG</a></td><td class="right " data-stat="age">34</td><td class="left " data-stat="pos">QB</td><td class="right " data-stat="g">16</td><td class="right " data-stat="gs">16</td><td class="right " csk="0.25000" data-stat="qb_rec">4-12-0</td><td class="right " data-stat="pass_cmp">437</td><td class="right " data-stat="pass_att">661</td><td class="right " data-stat="pass_cmp_perc">66.1</td><td class="right " data-stat="pass_yds">4792</td><td class="right " data-stat="pass_td">29</td><td class="right " data-stat="pass_td_perc">4.4</td><td class="right " data-stat="pass_int">13</td><td class="right " data-stat="pass_int_perc">2.0</td><td class="right " data-stat="pass_long">80</td><td class="right

The data we want for each player is found within the the `td` (or table data) elements.  

Below I've created a function that extracts the data we want from `table_rows`.  The comments should walk you through what each part of the function does.

In [9]:
def extract_player_data(table_rows):
    """
    Extract and return the the desired information from the td elements within
    the table rows.
    """
    # create the empty list to store the player data
    player_data = []
    
    for row in table_rows:  # for each row do the following

        # Get the text for each table data (td) element in the row
        # Some player names end with ' HOF', if they do, get the text excluding
        # those last 4 characters,
        # otherwise get all the text data from the table data
        player_list = [td.get_text() for td in row.find_all("td")]
        
        
        # there are some empty table rows, which are the repeated 
        # column headers in the table
        # we skip over those rows and and continue the for loop
        if not player_list:
            continue
            
        # Now append the data to list of data
        player_data.append(player_list)
        
    return player_data

Now we can create a `DataFrame` with the 2016 passing data

In [10]:
# extract the data we want
data = extract_player_data(table_rows)
#column_headers
# and then store it in a DataFrame
#df_2016 = pd.DataFrame(data, columns=column_headers)
df_2015  = pd.DataFrame(data)

In [11]:
data[1]

['Drew Brees',
 'NOR',
 '36',
 'QB',
 '15',
 '15',
 '7-8-0',
 '428',
 '627',
 '68.3',
 '4870',
 '32',
 '5.1',
 '11',
 '1.8',
 '80',
 '7.8',
 '8.0',
 '11.4',
 '324.7',
 '101.0',
 '75.47',
 '31',
 '235',
 '7.04',
 '7.26',
 '4.7',
 '1',
 '2']

In [12]:
df_2015.columns = ['Player_name']+column_headers

In [13]:
df_2015.head()

,Player_name,Tm,Age,Pos,G,GS,QBrec,Cmp,Att,Cmp%,...,Y/G,Rate,QBR,Sk,Yds,NY/A,ANY/A,Sk%,4QC,GWD
0,Philip Rivers,SDG,34,QB,16,16,4-12-0,437,661,66.1,...,299.5,93.8,59.44,40,264,6.46,6.45,5.7,1,2
1,Drew Brees,NOR,36,QB,15,15,7-8-0,428,627,68.3,...,324.7,101.0,75.47,31,235,7.04,7.26,4.7,1,2
2,Tom Brady*,NWE,38,QB,16,16,12-4-0,402,624,64.4,...,298.1,102.2,64.42,38,225,6.87,7.48,5.7,2,2
3,Eli Manning*,NYG,34,QB,16,16,6-10-0,387,618,62.6,...,277.0,93.6,60.46,27,157,6.63,6.74,4.2,1,2
4,Matt Ryan,ATL,30,QB,16,16,8-8-0,407,614,66.3,...,286.9,89.0,61.79,30,203,6.81,6.35,4.7,4,4


### Scraping the Data for All Seasons Since 1932

Scraping the for passing data since 1932 follows is essentially the same process as above, just repeated for each year, using a `for` loop.

As we loop over the years, we will create a `DataFrame` for each year of passing data, and append it to a large list of `DataFrame`s that contains all the passing data.  We will also have a separate list that will contain any errors and the url associated with that error.  This will let us know if there are any issues with our scraper, and which url is causing the error.

In [14]:
# Create an empty list that will contain all the dataframes
# (one dataframe for all passing dataframes)
passing_dfs_list = []

# a list to store any errors that may come up while scraping
errors_list = []

In [15]:
# The url template that we pass in the passing year info
url_template = "http://www.pro-football-reference.com/years/{year}/passing.htm"

# for each year from 1932 to (and including) 2016
for year in range(1932, 2017): 
    
    # Use try/except block to catch and inspect any urls that cause an error
    try:
        # get the passing data url
        url = url_template.format(year=year)

        # get the html
        html = urllib.request.urlopen(url)

        # create the BeautifulSoup object
        soup = BeautifulSoup(html, "lxml") 

        # get the column headers
        column_headers = [th.getText() for th in soup.findAll('th', limit=30)]

        column_headers = [s for s in column_headers if len(s) != 0]
        column_headers = column_headers[1:]
        
        
        #column_headers = ['Player_name']+column_headers
        #print(column_headers)
        

        # select the data from the table 
        table_rows = soup.find_all("tr")[1:]

        # extract the player data from the table rows
        player_data = extract_player_data(table_rows)
        #print((column_headers))
        # create the dataframe for the current year's passing data
        year_df = pd.DataFrame(player_data)

    
        # add the year of the pssing data to the dataframe
        year_df.insert(0, "Year", year)
   
        # append the current dataframe to the list of dataframes
        passing_dfs_list.append(year_df)
    
    except Exception as e:
        # Store the url and the error it causes in a list
        error =[url, e] 
        # then append it to the list of errors
        errors_list.append(error)

In [16]:
print(len(errors_list))
errors_list

0


[]

We don't get any errors, so that's good.

Now we can concatenate all the `DataFrame`s we scraped and create one large `DataFrame` containing all the passing data

In [17]:
type(passing_dfs_list)

list

In [18]:
passing_dfs_list[0:1]

[    Year                 0    1   2   3   4   5 6   7    8 ...    18    19  \
 0   1932     Arnie Herber+  GNB  22  tb  14   8    37  101 ...  17.3  45.6   
 1   1932       Walt Holmer  CRD  30   b  10   4    25   78 ...  18.0  44.9   
 2   1932    Benny Friedman  BKN  27  TB  11  10    23   74 ...  13.9  29.0   
 3   1932      Jack McBride  2TM  31      12   6    36   74 ...  10.1  30.3   
 4   1932  Keith Molesworth  CHI  27  QB  14  14    25   64 ...  13.8  24.7   
 5   1932   Honolulu Hughes  BOS  25  BB  10   9    11   57 ...  12.2  13.4   
 6   1932      Dutch Clark+  PRT  26  TB  11   9    17   52 ...  16.0  24.7   
 7   1932    Glenn Presnell  PRT  27  tb  12   7    17   46 ...  15.2  21.6   
 8   1932     Jack Grossman  BKN  22  FB  12   9    11   32 ...  16.2  14.8   
 9   1932      Jack Hagerty  NYG  29   b   4   4    17   32 ...   9.3  39.5   
 10  1932       Doug Wycoff  SIS  29  BB  12  11    10   31 ...  14.0  11.7   
 11  1932       Joe Lillard  CRD  27  TB   7   5    

In [19]:
column_headers.insert(0, "Player_name")
column_headers.insert(0, "Year")
print(column_headers)
print(len(column_headers))

['Year', 'Player_name', 'Tm', 'Age', 'Pos', 'G', 'GS', 'QBrec', 'Cmp', 'Att', 'Cmp%', 'Yds', 'TD', 'TD%', 'Int', 'Int%', 'Lng', 'Y/A', 'AY/A', 'Y/C', 'Y/G', 'Rate', 'QBR', 'Sk', 'Yds', 'NY/A', 'ANY/A', 'Sk%', '4QC', 'GWD']
30


In [20]:
# store all passing data in one DataFrame
passing_df = pd.concat(passing_dfs_list, axis=0)
#passing_df = passing_df.iloc[:,0:-1]
passing_df.columns = column_headers

In [21]:
# Take a look at the first few rows
passing_df.loc[0,:]

,Year,Player_name,Tm,Age,Pos,G,GS,QBrec,Cmp,Att,...,Y/G,Rate,QBR,Sk,Yds,NY/A,ANY/A,Sk%,4QC,GWD
0,1932,Arnie Herber+,GNB,22,tb,14,8,,37,101,...,45.6,51.5,,,,,,0,0,NaN
0,1933,Harry Newman,NYG,24,TB,14,12,,53,136,...,69.5,51.7,,,,,,,,NaN
0,1934,Arnie Herber,GNB,24,TB,11,7,,42,115,...,72.6,45.1,,,,,,0,0,NaN
0,1935,Ed Danowski+,NYG,24,TB,12,8,,57,113,...,66.2,69.7,,,,,,,,NaN
0,1936,Arnie Herber,GNB,26,TB,12,5,,77,173,...,103.3,58.9,,,,,,,,NaN
0,1937,Sammy Baugh+,WAS,23,TB,11,5,,81,171,...,102.5,50.5,,,,,,,,NaN
0,1938,Ace Parker+,BKN,26,TB,11,11,,63,148,...,78.6,53.5,,,,,,0,0,NaN
0,1939,Parker Hall*+,RAM,23,P/TB,11,11,,106,208,...,111.5,57.5,,,,,,0,0,NaN
0,1940,Davey O'Brien,PHI,23,QB,11,11,,124,277,...,117.3,39.2,,,,,,0,0,NaN
0,1941,Cecil Isbell*+,GNB,26,TB,11,4,,117,206,...,134.5,81.4,,,,,,,,NaN


Now that we fixed up the necessary columns, let's write out the raw data to a CSV file.

In [22]:
if not os.path.exists('../data/raw_data'):
    os.makedirs('../data/raw_data')

if not os.path.exists('../data/clean_data'):
    os.makedirs('../data/clean_data')

In [23]:
passing_df.head()

,Year,Player_name,Tm,Age,Pos,G,GS,QBrec,Cmp,Att,...,Y/G,Rate,QBR,Sk,Yds,NY/A,ANY/A,Sk%,4QC,GWD
0,1932,Arnie Herber+,GNB,22,tb,14,8,,37,101,...,45.6,51.5,,,,,,0,0,NaN
1,1932,Walt Holmer,CRD,30,b,10,4,,25,78,...,44.9,56.0,,,,,,0,0,NaN
2,1932,Benny Friedman,BKN,27,TB,11,10,,23,74,...,29.0,28.9,,,,,,0,0,NaN
3,1932,Jack McBride,2TM,31,,12,6,,36,74,...,30.3,50.5,,,,,,0,0,NaN
4,1932,Keith Molesworth,CHI,27,QB,14,14,,25,64,...,24.7,46.7,,,,,,0,0,NaN


In [24]:
# Write out the raw passing data to the raw_data fold in the data folder
passing_df.to_csv("../data/raw_data/pfr_nfl_passing_data_RAW.csv", index=False)

# Cleaning the Data

Now that we have the raw passing data, we need to clean it up a bit in order to do some of the data exploration we want.  

In [25]:
passing_df_clean = pd.read_csv("../data/raw_data/pfr_nfl_passing_data_RAW.csv")

* create a dictionary for renaming passing columns
* rename all columns with passing_columns dictionary

In [26]:
passing_columns = {
              'Year':'year', 'Player_name': 'name', 'Tm': 'team', 'Age': 'age',
              'Pos': 'position', 'G': 'games_played', 'GS':
              'games_started', 'QBrec': 'record', 'Cmp': 'completions',
              'Att': 'attempts', 'Cmp%': 'completionPct', 'Yds':
              'passing_yards', 'TD': 'passing_TD',  'TD%':
              'passing_TDPct', 'Int': 'passing_INT', 'Int%':
              'passing_INTPct', 'Lng': 'passing_long', 'Y/A':
              'passing_ydsAtt', 'AY/A': 'passing_airydsAtt', 'Y/C':
              'passing_ydsComp', 'Y/G': 'passing_ydsGame', 'Rate':
              'passing_rating', 'QBR': 'QBR','Sk': 'passing_sacks', 'Yds.1':
              'passing_sacksyds','NY/A': 'netydsAtt', 'ANY/A': 'passing_airnetydsAtt',
              'Sk%': 'passing_sackPct', '4QC': 'FourthQtrComebacks',
              'GWD': 'gamewinningdrives'
              }
ordered_columns = [
              'year', 'name', 'team', 'age', 'position', 'wins',
              'losses', 'games_played', 'games_started', 'completions',
              'attempts', 'completionPct', 'passing_yards',
              'passing_TD', 'passing_TDPct',  'passing_INT',
              'passing_INTPct', 'passing_long', 'passing_ydsAtt',
              'passing_airydsAtt', 'passing_ydsComp',
              'passing_ydsGame', 'passing_rating', 'passing_sacks',
              'passing_sacksyds', 'passing_airnetydsAtt',
              'passing_sackPct', 'FourthQtrComebacks',
              'gamewinningdrives'
    ]

In [27]:
passing_df_clean.rename(index=str,columns=passing_columns, inplace=True)
passing_df_clean.head()

,year,name,team,age,position,games_played,games_started,record,completions,attempts,...,passing_ydsGame,passing_rating,QBR,passing_sacks,passing_sacksyds,netydsAtt,passing_airnetydsAtt,passing_sackPct,FourthQtrComebacks,gamewinningdrives
0,1932,Arnie Herber+,GNB,22.0,tb,14,8,NaN,37,101,...,45.6,51.5,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN
1,1932,Walt Holmer,CRD,30.0,b,10,4,NaN,25,78,...,44.9,56.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN
2,1932,Benny Friedman,BKN,27.0,TB,11,10,NaN,23,74,...,29.0,28.9,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN
3,1932,Jack McBride,2TM,31.0,NaN,12,6,NaN,36,74,...,30.3,50.5,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN
4,1932,Keith Molesworth,CHI,27.0,QB,14,14,NaN,25,64,...,24.7,46.7,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN


## Cleaning Up the Rest of the Passing Data

* process record columns into wins and losses

In [28]:
passing_df_clean['record'] = passing_df_clean['record'].astype('str')
passing_df_clean['position'] = passing_df_clean['position'].astype('str')
passing_df_clean['position'] = passing_df_clean['position'].str.upper()
passing_df_clean['record'].loc[passing_df_clean.record == 'QBrec'] = "0-0-0"
passing_df_clean['record'].loc[passing_df_clean.record == 'nan'] = "0-0-0"
unformatted_record = passing_df_clean['record'].str[:]
unformatted_record = unformatted_record.str.replace("-",  "/")
passing_df_clean['wins'] = unformatted_record.str.split("/").str[0]
passing_df_clean['losses'] = unformatted_record.str.split("/").str[1]
passing_df_clean['name'] = passing_df_clean['name'].str.replace('[+|*]', "")
passing_df_clean['wins'] = passing_df_clean['wins'].astype('float')
passing_df_clean = pd.DataFrame(data=passing_df_clean, columns=ordered_columns)

/Users/kivan/anaconda3/lib/python3.5/site-packages/pandas/core/indexing.py:132: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [29]:
# convert the data to proper numeric types
passing_df_clean = passing_df_clean.apply(pd.to_numeric, errors="ignore")
passing_df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6955 entries, 0 to 6954
Data columns (total 29 columns):
year                    6955 non-null int64
name                    6955 non-null object
team                    6955 non-null object
age                     6951 non-null float64
position                6955 non-null object
wins                    6955 non-null float64
losses                  6955 non-null int64
games_played            6955 non-null int64
games_started           6955 non-null int64
completions             6955 non-null int64
attempts                6955 non-null int64
completionPct           6955 non-null float64
passing_yards           6955 non-null int64
passing_TD              6955 non-null int64
passing_TDPct           6955 non-null float64
passing_INT             6955 non-null int64
passing_INTPct          6955 non-null float64
passing_long            6955 non-null int64
passing_ydsAtt          6955 non-null float64
passing_airydsAtt       6955 non-null float64
p

We are not done yet. A lot of out numeric columns are missing data because players didn't accumulate any of those stats.  For example, some players didn't score a TD or even play a game.  Let's select the columns with numeric data and then replace the `NaN`s (the current value that represents the missing data) with 0s, as that is a more appropriate value. 

In [30]:
# Get the column names for the numeric columns
num_cols = passing_df_clean.columns[passing_df_clean.dtypes != object]

# Replace all NaNs with 0
passing_df_clean.loc[:, num_cols] = passing_df_clean.loc[:, num_cols].fillna(0)

In [31]:
passing_df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6955 entries, 0 to 6954
Data columns (total 29 columns):
year                    6955 non-null int64
name                    6955 non-null object
team                    6955 non-null object
age                     6955 non-null float64
position                6955 non-null object
wins                    6955 non-null float64
losses                  6955 non-null int64
games_played            6955 non-null int64
games_started           6955 non-null int64
completions             6955 non-null int64
attempts                6955 non-null int64
completionPct           6955 non-null float64
passing_yards           6955 non-null int64
passing_TD              6955 non-null int64
passing_TDPct           6955 non-null float64
passing_INT             6955 non-null int64
passing_INTPct          6955 non-null float64
passing_long            6955 non-null int64
passing_ydsAtt          6955 non-null float64
passing_airydsAtt       6955 non-null float64
p

We are finally done cleaning the data and now we can save it to a CSV file.

In [32]:
passing_df_clean = passing_df_clean[pd.notnull(passing_df_clean['name'])]
passing_df_clean.sort_values('wins', ascending=False, axis=0, inplace=True)


In [33]:
passing_df_clean.to_csv("../data/clean_data/pfr_nfl_passing_data_CLEAN.csv", index=False)
passing_df_clean.head(10)

,year,name,team,age,position,wins,losses,games_played,games_started,completions,...,passing_airydsAtt,passing_ydsComp,passing_ydsGame,passing_rating,passing_sacks,passing_sacksyds,passing_airnetydsAtt,passing_sackPct,FourthQtrComebacks,gamewinningdrives
6039,2007,Tom Brady,NWE,30.0,QB,16.0,0,16,16,398,...,9.4,12.1,300.4,117.2,21.0,128.00,8.88,3.5,4.0,4.0
6833,2015,Cam Newton,CAR,26.0,QB,15.0,1,16,16,296,...,8.3,13.0,239.8,99.4,33.0,284.00,7.20,6.3,3.0,4.0
4415,1991,Mark Rypien,WAS,29.0,QB,14.0,2,16,16,249,...,8.6,14.3,222.8,97.9,59.0,8.19,1.60,2.0,5.0,0.0
5610,2003,Tom Brady,NWE,26.0,QB,14.0,2,16,16,317,...,6.7,11.4,226.3,85.9,219.0,6.08,5.70,4.0,7.0,0.0
5952,2006,Philip Rivers,SDG,25.0,QB,14.0,2,16,16,284,...,7.4,11.9,211.8,92.0,27.0,144.00,6.73,5.5,4.0,4.0
3875,1986,Phil Simms,NYG,31.0,QB,14.0,2,16,16,259,...,6.2,13.5,217.9,74.6,359.0,6.10,8.80,2.0,4.0,0.0
4317,1990,Joe Montana,SFO,34.0,QB,14.0,1,15,15,321,...,7.2,12.3,262.9,89.0,153.0,6.91,5.30,3.0,5.0,0.0
5854,2005,Peyton Manning,IND,29.0,QB,14.0,2,16,16,305,...,8.5,12.3,234.2,104.1,81.0,7.80,3.60,1.0,1.0,0.0
3646,1984,Dan Marino,MIA,23.0,QB,14.0,2,16,16,362,...,9.4,14.0,317.8,108.9,120.0,8.60,2.30,2.0,3.0,0.0
3655,1984,Joe Montana,SFO,28.0,QB,14.0,1,16,15,279,...,8.7,13.0,226.9,102.9,138.0,7.69,4.80,2.0,2.0,0.0
